# Notebook de ejecucion de los algoritmos 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import json
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import model_from_json
from datetime import timedelta

import statsmodels.formula.api as smf

from src.dynamic_pricing_data_loader import cargar_y_preparar_datos

from Tools.ExtractData import GetDataFromETL
from Tools.Tools4TrainRN import RedNeuronal
from Tools.Tools4ProofRN import ProofRedNeuronal
from Tools.Tools4Elasticity import Elasticity
from Tools.Tools4Clasify import K_means
from Tools.Tools4ClasSuper import ClusteringSupervisado
from Tools.Tools4ProofCluster import PredictorClusters

# Clase para extracción de los datos

In [2]:
Data= GetDataFromETL(True)

El archivo JSON ha sido actualizado con las nuevas rutas.
Memoria usada antes: 5201.17 MB
Memoria usada después: 1001.59 MB
Reducción: 80.7%


## Data para ejecutar la red neuronal

In [3]:
Df_PRN= Data.D4_Proof_NN() # Extraccion de datos para probar la Red Neuronal

## Data para ejecutar el clustering supervisado

In [4]:
# clustering supervisado
Df_ClusSuper= Data.D4_ClusteringSuper()

## Ejecución del algoritmo de red neuronal

In [5]:
# Dataframe de pronosticos de nuevos clientes
lista_resultados = []
Df_=Df_PRN.copy()
for row in range(len(Df_PRN)):
    InfoClient = pd.DataFrame(Df_.iloc[row]).T
    PRN=ProofRedNeuronal(InfoClient)# Clase para pronosticar nuevos con la red
    InfoClient["TARIFA DINAMICA"]= PRN.PrecioDin[0,0]
    lista_resultados.append(InfoClient)

DataTD = pd.concat(lista_resultados, ignore_index=True)

## Ejecución del algoritmo de aprendizaje supervisado

In [6]:
# Entrena modelo ClusteringSupervisado
clustering = ClusteringSupervisado()
# Crear predictor con el modelo entrenado
predictor = PredictorClusters(Df_ClusSuper.copy())
resultados= predictor.Get_values()


Modelo XGBoost guardado exitosamente como: modelo_xgboost_clientes.json
Datos recibidos: 61259 filas, 8 columnas
Clientes a predecir: 37270 emails únicos
Columnas para el modelo: 25 features
Modelo espera 21 características
Columnas alineadas con el modelo entrenado
Predicciones de cluster realizadas


In [7]:
TD= DataTD[['NOMBRE_PASAJERO','FECHA_CORRIDA','HORA_SALIDA_CORRIDA','FECHA_OPERACION','HORA_OPERACION',
            'TIPO_PASAJERO','TARIFA DINAMICA']].copy()

In [8]:
TD.head(7)

,NOMBRE_PASAJERO,FECHA_CORRIDA,HORA_SALIDA_CORRIDA,FECHA_OPERACION,HORA_OPERACION,TIPO_PASAJERO,TARIFA DINAMICA
0,TANATIUH OCAMPO POPCA,2025-10-20,1900-01-01 02:00:00,2025-10-20 00:00:00,1900-01-01 00:00:27,AD,1053.811401
1,MARIA DE JESUS VARGAS HERNANDEZ,2025-10-20,1900-01-01 00:30:00,2025-10-20 00:00:00,1900-01-01 00:12:09,AD,1057.222900
2,ANILU DIAZ LUNA,2025-10-20,1900-01-01 00:30:00,2025-10-20 00:00:00,1900-01-01 00:17:08,AD,1057.172852
3,ARIADNE ARLETTE CASTILLO DUARTE,2025-10-20,1900-01-01 00:30:00,2025-10-20 00:00:00,1900-01-01 00:24:40,AD,1055.584595
4,JORDY ATANAEL MORALE ROJAS,2025-10-20,1900-01-01 02:00:00,2025-10-20 00:00:00,1900-01-01 00:50:39,PE,1053.461670
5,MARIA JULIA AGULERA VEGA,2025-10-20,1900-01-01 02:00:00,2025-10-20 00:00:00,1900-01-01 00:58:26,AD,1053.661621
6,JESUS JAIMES CRUZ,2025-10-20,1900-01-01 02:00:00,2025-10-20 00:00:00,1900-01-01 01:27:08,AD,1055.024048


In [9]:
TP= resultados[['NOMBRE_PASAJERO','FECHA_CORRIDA','HORA_SALIDA_CORRIDA','FECHA_OPERACION','HORA_OPERACION',
            'TIPO_PASAJERO','ORIGEN_DESTINO','BOLETOS_VEND','EMAIL','Cluster','Aumento']].copy()

In [10]:
TP[['NOMBRE_PASAJERO','FECHA_CORRIDA','HORA_SALIDA_CORRIDA','TIPO_PASAJERO','ORIGEN_DESTINO','EMAIL','Cluster','Aumento']].head(7)

,NOMBRE_PASAJERO,FECHA_CORRIDA,HORA_SALIDA_CORRIDA,TIPO_PASAJERO,ORIGEN_DESTINO,EMAIL,Cluster,Aumento
0,JESUS JAIMES CRUZ,2025-10-20,1900-01-01 02:00:00,AD,MEXN-GDLJ,TAQUILLAGDLJ@GMAIL.COM,1,3
1,QUINTINA RIOS FIERROS,2025-10-21,1900-01-01 22:00:00,AD,GDLJ-MEXN,LUISMATEOSCATEDRAL@GMAIL.COM,3,13
2,JOSÉ MORA VELÁZQUEZ,2025-10-21,1900-01-01 22:00:00,OD,GDLJ-MEXN,LUISMATEOSCATEDRAL@GMAIL.COM,3,13
3,SONIA LIBIEHT MORA RIOS,2025-10-21,1900-01-01 22:00:00,OD,GDLJ-MEXN,LUISMATEOSCATEDRAL@GMAIL.COM,3,13
4,STEFAN GANDLER,2025-11-17,1900-01-01 07:00:00,IN,MXST-QUER,STEFAN.GANDLER@GMAIL.COM,2,7
5,SEBASTIAN FLORES ALVARADO,2025-10-20,1900-01-01 20:50:00,AD,ZACA-AGUA,SEBASTIANFLORES.MTB@GMAIL.COM,2,7
6,ZAID GONZALEZ,2025-10-20,1900-01-01 07:10:00,AD,LEON-MEXN,ZGONZALEZ@ACCORDIAUSA.COM,3,13


In [11]:
len(TD),len(TP)

(87341, 87341)

In [12]:
len(Df_PRN),len(Df_ClusSuper)

(87341, 87341)

In [13]:
# Definimos las columnas que determinan el orden único
columnas_orden = [
    'NOMBRE_PASAJERO', 
    'FECHA_CORRIDA', 
    'HORA_SALIDA_CORRIDA', 
    'FECHA_OPERACION', 
    'HORA_OPERACION',
    'TIPO_PASAJERO'
]

# Aplicamos el ordenamiento a TD
TD = TD.sort_values(by=columnas_orden).reset_index(drop=True)

# Aplicamos el mismo ordenamiento a TP
TP = TP.sort_values(by=columnas_orden).reset_index(drop=True)

# Aplicamos el mismo ordenamiento a Df_PRN
Df_PRN1=Df_PRN.sort_values(by=columnas_orden).reset_index(drop=True)
# Aplicamos el mismo ordenamiento a Df_ClusSuper
Df_ClusSuper1=Df_ClusSuper.sort_values(by=columnas_orden).reset_index(drop=True)

TD['VENTA']=Df_PRN1['VENTA'].copy()
df_final=TD.copy()
df_final[['ORIGEN_DESTINO','EMAIL','BOLETOS_VEND','Cluster','Aumento']]=TP[['ORIGEN_DESTINO','EMAIL','BOLETOS_VEND','Cluster','Aumento']].copy()

acla_keys = ['PF', 'AD', 'ES', 'IN', 'MA', 'NI', 'PD', 'PE', 'XA', 'ZA', 'ZB', 'VC', 'PH', 'FA', 'OD', 'XP', 'IF', 'SC']
porcentaje_values = [25, 0, 50, 50, 25, 50, 25, 0, 5, 0, 10, 25, 6, 0, 50, 25, 0, 0]

# Crear el diccionario (Mapeo de código a porcentaje entero)
diccionario_acla_porcentaje = dict(zip(acla_keys, porcentaje_values))

# 1. Mapea el TIPO_PASAJERO al porcentaje entero (25, 50, 0, etc.)
# 2. Divide entre 100 para convertirlo al factor decimal (0.25, 0.50, 0.0)
df_final['Descuento'] = df_final['TIPO_PASAJERO'].map(diccionario_acla_porcentaje) / 100
df_final['Tarifa_final']= df_final['TARIFA DINAMICA']*(1+df_final['Aumento']/100)
df_final['VentaD_final']= df_final['Tarifa_final']*(1-df_final['Descuento'])
df_final['Delta_P']=(df_final['VentaD_final']-df_final['VENTA'])/df_final['VentaD_final']

In [14]:
df_final[['NOMBRE_PASAJERO', 'FECHA_CORRIDA','HORA_SALIDA_CORRIDA', 
'TIPO_PASAJERO','ORIGEN_DESTINO','EMAIL','Cluster','TARIFA DINAMICA','Aumento','VentaD_final']].to_excel('Resumen.xlsx', index=False)

In [103]:
df_comp= df_final.groupby(['ORIGEN_DESTINO','FECHA_CORRIDA']).agg({
    'BOLETOS_VEND':'sum',
    'VentaD_final':'sum',
    'VENTA':'sum'
}).reset_index()

df_comp['Diff_P_Dinero']= df_comp['VentaD_final']-df_comp['VENTA']
df_comp['%dP']=(df_comp['VENTA']-df_comp['VentaD_final'])/df_comp['VentaD_final']
df_comp['Precio_Prom']=df_comp['VENTA']/df_comp['BOLETOS_VEND']
df_comp['Precio_Prom_Dinamico']=df_comp['VentaD_final']/df_comp['BOLETOS_VEND']

In [104]:
Elas= pd.read_excel('Elas_2026.xlsx')

In [105]:
# 1. Convertir a datetime por si acaso
df_comp['FECHA_CORRIDA'] = pd.to_datetime(df_comp['FECHA_CORRIDA'])
Elas['FECHA_CORRIDA'] = pd.to_datetime(Elas['FECHA_CORRIDA'])

# 2. Crear columnas temporales de Mes y Día en ambos dataframes
df_comp['mes'] = df_comp['FECHA_CORRIDA'].dt.month
df_comp['dia'] = df_comp['FECHA_CORRIDA'].dt.day

Elas['mes'] = Elas['FECHA_CORRIDA'].dt.month
Elas['dia'] = Elas['FECHA_CORRIDA'].dt.day
# 3. Unir los dataframes
# Suponiendo que la columna con el valor se llama 'valor_elasticidad' en Elas
df_comp = df_comp.merge(
    Elas[['ORIGEN_DESTINO', 'mes', 'dia', 'ELASTICIDADES']], 
    on=['ORIGEN_DESTINO', 'mes', 'dia'], 
    how='left'
)

# 4. Renombrar la columna resultante a 'Elast'
df_comp = df_comp.rename(columns={'ELASTICIDADES': 'Elast'})

# 5. (Opcional) Eliminar las columnas auxiliares
df_comp = df_comp.drop(columns=['mes', 'dia'])

In [106]:
df_comp['%dQ']=df_comp['%dP']*df_comp['Elast']
df_comp['Boletos_Estimados']=df_comp['BOLETOS_VEND']*(df_comp['Elast']+1)
df_comp['Ingreso_Estimado']=df_comp['Boletos_Estimados']*df_comp['Precio_Prom_Dinamico']
df_comp['Diff_P_Dinero_Estimado']= df_comp['Ingreso_Estimado']-df_comp['VENTA']
df_comp['%dP_Estimado']=(df_comp['Ingreso_Estimado']-df_comp['VENTA'])/df_comp['Ingreso_Estimado']
df_comp['%dQ_Estimado']=(df_comp['Boletos_Estimados']-df_comp['BOLETOS_VEND'])/df_comp['Boletos_Estimados']

In [107]:
df_comp

,ORIGEN_DESTINO,FECHA_CORRIDA,BOLETOS_VEND,VentaD_final,VENTA,Diff_P_Dinero,%dP,Precio_Prom,Precio_Prom_Dinamico,Elast,%dQ,Boletos_Estimados,Ingreso_Estimado,Diff_P_Dinero_Estimado,%dP_Estimado,%dQ_Estimado
0,GDLJ-MEXN,2025-10-20,124.0,123972.447341,138023.390625,-14050.943284,0.113339,1113.091919,999.777801,0.813590,0.092212,224.885203,224835.233732,86811.843107,0.386113,0.448608
1,GDLJ-MEXN,2025-10-21,127.0,126498.109997,135964.593750,-9466.483753,0.074835,1070.587402,996.048110,0.893827,0.066890,240.516079,239565.586156,103600.992406,0.432454,0.471969
2,GDLJ-MEXN,2025-10-22,157.0,159659.906014,171564.078125,-11904.172111,0.074560,1092.764771,1016.942077,0.845309,0.063026,289.713562,294621.911750,123057.833625,0.417681,0.458085
3,GDLJ-MEXN,2025-10-23,309.0,317587.305677,341231.187500,-23643.881823,0.074448,1104.308105,1027.790633,0.633270,0.047146,504.680486,518705.876040,177474.688540,0.342149,0.387731
4,GDLJ-MEXN,2025-10-24,319.0,326777.398035,351836.156250,-25058.758215,0.076684,1102.934692,1024.380558,0.306976,0.023540,416.925399,427090.273385,75254.117135,0.176202,0.234875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,ZACA-AGUA,2026-01-24,14.0,3513.116099,3944.100098,-430.983999,0.122679,281.721436,250.936864,0.987504,0.121146,27.825061,6982.333507,3038.233410,0.435132,0.496856
885,ZACA-AGUA,2026-01-25,8.0,2108.717395,2392.360107,-283.642712,0.134510,299.045013,263.589674,0.988452,0.132956,15.907615,4193.082980,1800.722873,0.429451,0.497096
886,ZACA-AGUA,2026-01-26,9.0,2432.524158,2699.520020,-266.995862,0.109761,299.946655,270.280462,0.995193,0.109233,17.956733,4853.354205,2153.834186,0.443783,0.498795
887,ZACA-AGUA,2026-01-27,13.0,3014.184668,3342.699951,-328.515284,0.108990,257.130768,231.860359,0.992637,0.108187,25.904284,6006.176485,2663.476534,0.443456,0.498152


In [108]:
df_comp.to_excel('DatosDinamicos.xlsx', index=False)

In [109]:
df_comp.groupby(['ORIGEN_DESTINO']).agg({
    '%dP_Estimado':'min'
}).reset_index().to_excel('PorcentajeCrecimientoPorRuta.xlsx', index=False)

In [110]:
df_comp.groupby(['ORIGEN_DESTINO']).agg({
    '%dQ_Estimado':'min'
}).reset_index().to_excel('PorcentajeBolCrecimientoPorRuta.xlsx', index=False)